In [1]:
import keras
from keras import ops
from keras import layers

# IMPLEMENTING TRANSFORMER BLOCK

In [11]:
class TransformerBlock(layers.Layer):
  # will contain all our layers that happens insider the block

    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
      #embed dim : size of EMbedding vector
      # numhead - number of multi head attentions
      # ff -> feed forward neural network
      # rate -> for dropout
        super().__init__()
        #multi head Attention layer
        self.att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
       # nn
        self.ffn = keras.Sequential(
            [layers.Dense(ff_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        #2 layer normalization
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        #dropout to be used later
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs):
      # apply multi head self attention
        attn_output = self.att(inputs, inputs)
        # apply droput in it
        attn_output = self.dropout1(attn_output)
        # adding layer Normalizatin
        out1 = self.layernorm1(inputs + attn_output)
        #neural netword with dropout
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output)
        # layer normalization
        return self.layernorm2(out1 + ffn_output)

Embedding and TOkenization

In [12]:
class TokenAndPositionEmbedding(layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super().__init__()
        # learning new token embedding
        self.token_emb = layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
        #position encoding -> sequence in the token encoding
        self.pos_emb = layers.Embedding(input_dim=maxlen, output_dim=embed_dim)

    def call(self, x):
        maxlen = ops.shape(x)[-1]
        positions = ops.arange(start=0, stop=maxlen, step=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions
        #convert token IDS to token vectors

# Data Ingestion

In [13]:
vocab_size = 20000
#max length of sentence=200
maxlen = 200


(x_train, y_train), (x_val, y_val) = keras.datasets.imdb.load_data(num_words=vocab_size)
print(len(x_train), "Training sequences")
print(len(x_val), "Validation sequences")


25000 Training sequences
25000 Validation sequences


In [14]:
#PADDING

x_train = keras.utils.pad_sequences(x_train, maxlen=maxlen)
x_val = keras.utils.pad_sequences(x_val, maxlen=maxlen)

# MODEL

In [15]:
embed_dim=32 # embedding size for each token
num_heads=2 # nom of attention heads
ff_dim=32 # 32 hidden layer size in feed forward network

In [16]:
inputs=layers.Input(shape=(maxlen,))
embedding_layer=TokenAndPositionEmbedding(maxlen,vocab_size,embed_dim)
#setting embedding layer

In [17]:
import tensorflow as tf

In [18]:
x = embedding_layer(inputs)
# applying token and position Embedding layer
#converted words into token then positional tokens.
transformer_block = TransformerBlock(embed_dim, num_heads, ff_dim)
x = transformer_block(x)
#applying transformer block
x = layers.GlobalAveragePooling1D()(x)
#reduce sequence to single vector per sample
x = layers.Dropout(0.1)(x)
#nn
x = layers.Dense(20, activation="relu")(x)
x = layers.Dropout(0.1)(x)
#predicting the 1 word
outputs = layers.Dense(2, activation="softmax")(x)

model = keras.Model(inputs=inputs, outputs=outputs)

In [19]:
model.compile(optimizer="adam",loss="sparse_categorical_crossentropy",metrics=["accuracy"])



In [10]:
history=model.fit(    x_train, y_train, batch_size=32, epochs=2, validation_data=(x_val, y_val))

Epoch 1/2
782/782 ━━━━━━━━━━━━━━━━━━━━ 66s 80ms/step - accuracy: 0.6945 - loss: 0.5385 - val_accuracy: 0.8798 - val_loss: 0.2849
Epoch 2/2
782/782 ━━━━━━━━━━━━━━━━━━━━ 61s 78ms/step - accuracy: 0.9274 - loss: 0.1945 - val_accuracy: 0.8739 - val_loss: 0.3019
